In [1]:
%load_ext autoreload
%autoreload 2
from data_translator import CMAToScheduleTranslator, TestTranslator, EncodeForCMA, EncodeForGA, GAToScheduleTranslator
from hybrid_solution_data_loader import get_data
from models import Order, SimulationEnvironment, Schedule
from solver import CMAWeightSolver, HybridGreedyAgentSolver
from visualize import visualize_schedule_demo
import objective_function

In [2]:
n_workstations, recipes, operation_times = get_data(3)
recipies, workstations, resources, tasks, _ = TestTranslator().translate(n_workstations, recipes, operation_times)

env = SimulationEnvironment(workstations, tasks, resources, recipies)

In [3]:
earliest_slot = 0
last_slot = 100
recipe_orders = [0, 1, 2, 3, 0, 3, 2, 1, 0, 3] # for dataset 0
orders = []
o_id = 0
for order in recipe_orders:
    orders.append(Order(o_id, 0, last_slot, last_slot, [order], 100, 50, False, 0, False, 500)) # for now: use resources to select recipe
    o_id = o_id + 1

In [4]:
encoder : EncodeForCMA = EncodeForCMA()
values, durations, jobs, due_dates = encoder.translate(env, orders)

In [5]:
solver : CMAWeightSolver = CMAWeightSolver(values, durations, jobs, env, orders)
solver.initialize() # doesn't do anything right now
solver.run()

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 40 (seed=155022, Wed Jan 18 09:06:20 2023)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 2.000000000000000e+00 1.0e+00 4.69e-01  5e-01  5e-01 0:02.5
    2     30 2.000000000000000e+00 1.0e+00 4.51e-01  4e-01  5e-01 0:04.8
    3     45 2.000000000000000e+00 1.1e+00 4.42e-01  4e-01  4e-01 0:07.1
    5     75 2.000000000000000e+00 1.1e+00 4.35e-01  4e-01  4e-01 0:11.6
    7    105 2.000000000000000e+00 1.1e+00 4.39e-01  4e-01  5e-01 0:16.0
    8    120 2.000000000000000e+00 1.1e+00 4.38e-01  4e-01  4e-01 0:18.1
termination on tolflatfitness=1 (Wed Jan 18 09:06:38 2023)
final/bestever f-value = 2.000000e+00 2.000000e+00
incumbent solution: [0.11126619 0.31154961 1.06761676 0.00977598 0.23389311 0.34172128
 0.47909694 0.49034178 ...]
std deviations: [0.43324512 0.43247434 0.43869114 0.43586477 0.44122406 0.43831211
 0.43683171 0.43803321 ...]
Done


In [6]:
result = solver.best_solution[0]
print("Parameters of the best solution :\n {solution}".format(solution=result))
print(f'Fitness value of the solution: {solver.best_solution[1].result.fbest}')

Parameters of the best solution :
 [0.75471799 0.46053623 0.00943063 0.178834   0.16219971 0.44138252
 0.7746765  0.28258172 0.24716997 0.05364336 0.09500606 0.22188406
 0.363538   0.5348705  0.09330197 0.07571869 0.20082664 0.43466254
 0.16793987 0.07147676 0.28250112 0.52939905 0.00286458 0.04595927
 0.33690277 0.33625276 0.27538725 0.26052737 0.14331457 0.800976
 0.4687773  0.06471367 0.02727973 1.13255006 0.41294627 0.73100518
 0.05707609 0.01958411 0.11192072 0.72669171]
Fitness value of the solution: 2


In [7]:
schedule = CMAToScheduleTranslator().translate(result, jobs, due_dates, durations, env, orders)
schedule.created_in = env
schedule.created_for = orders
schedule.created_by = solver

In [8]:
#TODO: current solution could be a result of either a) bug in schedule building (more likely) or b) lack of feasibility checks, since they shouldn't be necessary
visualize_schedule_demo(schedule, env, orders)

In [9]:
fitness_values = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {fitness_values[0]}\nTardiness: {fitness_values[1]}\nDeviation: {fitness_values[2]}\nIdle Time: {fitness_values[3]}\nProfit: {fitness_values[4]}')

AttributeError: 'NoneType' object has no attribute 'tasks'

In [10]:
values = [9,2,3,4,5,1,1,2,4,5,6,7,6,2,2,3,4,5,5,6]
sorted = []
for i in range(len(values)):
    idx = values.index(min(values)) # workstation id
    sorted.append(values[idx])
    values[idx] = float('inf')
print(sorted)
print(values)

[1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 9]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]


In [39]:
encoder = EncodeForGA()
values, durations, all_jobs = encoder.translate(env, orders)

     
crossover = 'two_points' #NOTE: available in PyGAD: 'two_points', 'single_point', 'uniform', 'scattered'
selection = 'rws' #NOTE: available in PyGAD: 'sss' (steady state selection', 'rws' (roulette wheel), 'sus' (stochastic universal selection), 'rank' (rank selection), 'random' (random selection), 'tournament' (tournament selection)
mutation = 'inversion' #NOTE: available options: 'swap', 'inversion', 'scramble', 'adaptive
#objective = 'makespan' #NOTE: available options: 'makespan', 'idle_time'
population_size = 50
max_generations = 2000

solver = HybridGreedyAgentSolver(values, durations, all_jobs, env, orders)
solver.initialize(mutation, crossover, selection, max_generations, population_size)
solver.run()

Done
Done


In [40]:
result = solver.get_best()
fitness = solver.get_best_fitness()

print("Parameters of the best solution : {solution}".format(solution=result))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(fitness)))

Parameters of the best solution : [2, 0, 2, 1, 2, 2, 2, 3, 9, 0, 9, 2, 2, 4, 2, 5, 0, 0, 9, 4, 2, 6, 2, 7, 1, 0, 9, 6, 2, 8, 2, 9, 0, 2, 9, 8, 2, 10, 2, 11, 3, 0, 3, 5, 9, 10, 9, 12, 0, 4, 0, 6, 0, 8, 0, 10, 7, 0, 0, 12, 0, 14, 0, 16, 6, 0, 6, 5, 9, 14, 9, 16, 4, 0, 2, 13, 2, 14, 2, 15]
Fitness value of the best solution = 18


In [49]:
new_result = solver.re_run_agent()
print("Parameters of the best solution : {solution}".format(solution=new_result[0]))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(new_result[1])))

Done
Parameters of the best solution : [2, 0, 2, 1, 2, 2, 2, 3, 9, 0, 9, 2, 2, 4, 2, 5, 8, 0, 9, 4, 2, 6, 2, 7, 0, 0, 8, 3, 9, 6, 2, 8, 3, 0, 8, 6, 2, 9, 2, 10, 3, 5, 2, 12, 2, 13, 2, 14, 0, 2, 0, 4, 0, 6, 3, 10, 9, 8, 9, 10, 9, 12, 9, 14, 1, 0, 2, 15, 2, 16, 2, 17, 5, 0, 8, 9, 9, 16, 2, 18]
Fitness value of the best solution = 19


In [50]:
result = new_result[0]
fitness = new_result[1]

In [51]:
schedule = GAToScheduleTranslator().translate(result, all_jobs, env, orders)
schedule.created_in = env
schedule.created_for = orders
visualize_schedule_demo(schedule, env, orders)
fitness_values = objective_function.calculate_comparison_values(schedule, orders, env)
print(f'Fitness Values:\nMakespan: {fitness_values[0]}\nTardiness: {fitness_values[1]}\nDeviation: {fitness_values[2]}\nIdle Time: {fitness_values[3]}\nProfit: {fitness_values[4]}')

Fitness Values:
Makespan: 23
Tardiness: 0
Deviation: 859
Idle Time: -13
Profit: 5000


In [55]:
import pickle

#solver.environment = None # make file smaller, since the schedule already knows the environment and the orders
#solver.orders = None
#schedule.created_by = solver
schedule.created_by = 'HybridAgentSolver'
schedule.evaluation_results = []
schedule.evaluation_results.append(('makespan', fitness_values[0]))
schedule.evaluation_results.append(('tardiness', fitness_values[1]))
schedule.evaluation_results.append(('deviation', fitness_values[2]))
schedule.evaluation_results.append(('idle_time', fitness_values[3]))
schedule.evaluation_results.append(('profit', fitness_values[4]))
save_file = True
if(save_file):
    with open(f'results/HybridSolverTesting.pickle', 'wb') as outfile:
        pickled_data = pickle.dump(schedule, outfile)